In [3]:
# B3_Configurable_Minute_SOL_Optuna.py
# Predictor with Sliding Window, Per-Step HParam Tuning (Optuna), and PTT
# Adapted for minute-level data from SOL_minagg.csv (single symbol).

import pandas as pd
import numpy as np
import time
import os
import warnings
import traceback
from datetime import datetime
import xgboost as xgb
import matplotlib.pyplot as plt
import optuna # <--- ADDED: Optuna import

# Modeling Imports
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# GridSearchCV no longer needed
from sklearn.model_selection import StratifiedKFold # Keep StratifiedKFold for CV inside Optuna objective
from sklearn.exceptions import UndefinedMetricWarning

# --- Suppress Warnings ---
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings('ignore')
# Reduce Optuna logging verbosity
optuna.logging.set_verbosity(optuna.logging.WARNING)

# ==============================================================================
# --- Configuration ---
# ==============================================================================

# --- Data ---
CSV_FILE_PATH = 'SOL_minagg.csv'
SYMBOL_NAME = 'SOL'

# --- Target Definition ---
PREDICTION_WINDOW_MINUTES = 12 * 60
TARGET_THRESHOLD_PCT = 2.5

# --- Backtesting Windowing (Defined in MINUTES) ---
TRAIN_WINDOW_MINUTES = 24 * 60
STEP_MINUTES = 1 * 60
TEST_WINDOW_FRACTION = 0.2

# --- Model & Tuning ---

# Fixed parameters (always active for XGBoost)
XGB_FIXED_PARAMS = {
    "objective":        "binary:logistic",
    "eval_metric":      "logloss",
    "use_label_encoder": False,
    "random_state":      42,
    "tree_method":      "gpu_hist",     # Use GPU
    "predictor":        "gpu_predictor",# Use GPU
    "gpu_id":            0,
    "n_jobs":           -1,             # Allow XGBoost to use cores for host-side tasks
    # Some parameters previously in grid search can be fixed here if desired
    "n_estimators":     150,            # Fix n_estimators, tune learning_rate
    "reg_alpha":        0.1,            # Example fixed L1
    "reg_lambda":       4.0,            # Example fixed L2
    "gamma":            0.1,            # Example fixed gamma
}

# --- Optuna Configuration ---
N_OPTUNA_TRIALS = 50           # Number of trials Optuna runs per step
N_CV_FOLDS = 3                  # Number of folds for cross-validation within Optuna objective

# Probability Threshold Tuning Range
PROBABILITY_THRESHOLD_RANGE = (0.10, 0.90)
PROBABILITY_THRESHOLD_STEP = 0.05

# ==============================================================================
# --- Derived Variables (Do not change these directly) ---
# ==============================================================================
TEST_WINDOW_MINUTES = max(1, int(TEST_WINDOW_FRACTION * TRAIN_WINDOW_MINUTES))
THRESHOLD_SEARCH_RANGE = np.arange(
    PROBABILITY_THRESHOLD_RANGE[0],
    PROBABILITY_THRESHOLD_RANGE[1],
    PROBABILITY_THRESHOLD_STEP
)
# grid_combinations no longer directly relevant, Optuna uses N_OPTUNA_TRIALS

# ==============================================================================
# --- Feature Engineering Functions (Minute Based - Unchanged) ---
# ==============================================================================
def garman_klass_volatility_min(o, h, l, c, window_min):
    with np.errstate(divide='ignore', invalid='ignore'): log_hl=np.log(h/l.replace(0,np.nan)); log_co=np.log(c/o.replace(0,np.nan))
    gk = 0.5*(log_hl**2) - (2*np.log(2)-1)*(log_co**2); gk = gk.fillna(0)
    min_p = max(1, window_min // 4); rm = gk.rolling(window_min, min_periods=min_p).mean(); rm = rm.clip(lower=0); return np.sqrt(rm)
def parkinson_volatility_min(h, l, window_min):
    with np.errstate(divide='ignore', invalid='ignore'): log_hl_sq = np.log(h/l.replace(0,np.nan))**2
    log_hl_sq = log_hl_sq.fillna(0); min_p = max(1, window_min // 4); rs = log_hl_sq.rolling(window_min, min_periods=min_p).sum()
    f = 1/(4*np.log(2)*window_min) if window_min>0 else 0; return np.sqrt(f*rs)
def calculate_features_min(df_input):
    df = df_input.copy()
    base_cols_numeric = ['open', 'high', 'low', 'close', 'volumefrom', 'volumeto']
    for col in base_cols_numeric:
        if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce')
        else: print(f"Warning: Missing base column '{col}'"); df[col] = 0
    if df[['open', 'high', 'low', 'close']].isnull().any().any():
        df = df.dropna(subset=['open', 'high', 'low', 'close'])
    if df.empty: return df
    df['price_change_1m_temp'] = df['close'].pct_change(periods=1)
    with np.errstate(divide='ignore', invalid='ignore'):
        df['price_range_pct'] = (df['high'] - df['low']) / df['close'].replace(0, np.nan) * 100
        df['oc_change_pct'] = (df['close'] - df['open']) / df['open'].replace(0, np.nan) * 100
    df['garman_klass_720m'] = garman_klass_volatility_min(df['open'], df['high'], df['low'], df['close'], 12 * 60)
    df['parkinson_180m'] = parkinson_volatility_min(df['high'], df['low'], 3 * 60)
    min_periods_rolling = 2
    df['ma_180m'] = df['close'].rolling(3 * 60, min_periods=max(min_periods_rolling, (3*60)//4)).mean()
    df['rolling_std_180m'] = df['close'].rolling(3 * 60, min_periods=max(min_periods_rolling, (3*60)//4)).std()
    lag_periods_price_min = [3*60, 6*60, 12*60, 24*60, 48*60, 72*60, 168*60]
    lag_periods_volume_min = [3*60, 6*60, 12*60, 24*60]
    for lag in lag_periods_price_min: df[f'lag_{lag}m_price_return'] = df['price_change_1m_temp'].shift(lag) * 100
    df['volume_return_1m'] = df['volumefrom'].pct_change(periods=1) * 100
    for lag in lag_periods_volume_min: df[f'lag_{lag}m_volume_return'] = df['volume_return_1m'].shift(lag)
    ma_periods_min = [6*60, 12*60, 24*60, 48*60, 72*60, 168*60]
    std_periods_min = [6*60, 12*60, 24*60, 48*60, 72*60, 168*60]
    min_p_long = 50
    for p in ma_periods_min: df[f'ma_{p}m'] = df['close'].rolling(p, min_periods=max(min_p_long, p//4)).mean()
    for p in std_periods_min: df[f'rolling_std_{p}m'] = df['price_change_1m_temp'].rolling(p, min_periods=max(min_p_long, p//4)).std() * 100
    df['prev_close']=df['close'].shift(1); df['hml']=df['high']-df['low']; df['hmpc']=np.abs(df['high']-df['prev_close']); df['lmpc']=np.abs(df['low']-df['prev_close'])
    df['tr']=df[['hml','hmpc','lmpc']].max(axis=1)
    atr_periods_min = [14*60, 24*60, 48*60]; min_p_atr = 20
    for p in atr_periods_min: df[f'atr_{p}m'] = df['tr'].rolling(p, min_periods=max(min_p_atr, p//4)).mean()
    df = df.drop(columns=['prev_close', 'hml', 'hmpc', 'lmpc', 'tr'])
    epsilon = 1e-9
    for p in [24*60, 168*60]: mc=f'ma_{p}m'; df[f'close_div_ma_{p}m'] = df['close']/(df[mc]+epsilon) if mc in df else np.nan
    if 'ma_720m' in df and 'ma_2880m' in df: df['ma720_div_ma2880'] = df['ma_720m']/(df['ma_2880m']+epsilon)
    else: df['ma720_div_ma2880']=np.nan
    if 'ma_1440m' in df and 'ma_10080m' in df: df['ma1440_div_ma10080'] = df['ma_1440m']/(df['ma_10080m']+epsilon)
    else: df['ma1440_div_ma10080']=np.nan
    if 'rolling_std_720m' in df and 'rolling_std_4320m' in df: df['std720_div_std4320'] = df['rolling_std_720m']/(df['rolling_std_4320m']+epsilon)
    else: df['std720_div_std4320']=np.nan
    if 'price_range_pct' in df: df['volumefrom_x_range'] = df['volumefrom'] * df['price_range_pct']
    else: df['volumefrom_x_range']=np.nan
    if 'rolling_std_180m' in df: df['rolling_std_180m_sq'] = df['rolling_std_180m']**2
    else: df['rolling_std_180m_sq']=np.nan
    if 'price_change_1m_temp' in df: df['price_return_1m_sq'] = df['price_change_1m_temp']**2 * 10000
    else: df['price_return_1m_sq']=np.nan
    if 'rolling_std_720m' in df: df['rolling_std_720m_sqrt'] = np.sqrt(df['rolling_std_720m'].clip(lower=0)+epsilon)
    else: df['rolling_std_720m_sqrt']=np.nan
    cols_to_drop_intermediate = ['price_change_1m_temp', 'volume_return_1m']
    df = df.drop(columns=[col for col in cols_to_drop_intermediate if col in df.columns])
    return df

# ==============================================================================
# --- Optuna Objective Function ---
# ==============================================================================

def objective(trial, X_train, y_train, fixed_params, n_folds):
    """Objective function for Optuna hyperparameter optimization."""

    # Define the search space for this trial
    param_suggestions = {
        # Using suggest_float for continuous params allows finer exploration
        'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.15, log=True),
        'max_depth': trial.suggest_int('max_depth', 5, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        # Suggest scale_pos_weight based on potential imbalance.
        # If balanced, could fix this to 1. For potentially imbalanced, explore.
        'scale_pos_weight': trial.suggest_int('scale_pos_weight', 1, 5),
        # Can add back other parameters like reg_alpha, reg_lambda, gamma here if needed
        # 'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 1.0, log=True),
        # 'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0, log=True),
        # 'gamma': trial.suggest_float('gamma', 0, 0.5),
    }

    # Merge fixed and suggested parameters
    params = {**fixed_params, **param_suggestions}

    cv_scores = []
    # Use StratifiedKFold for cross-validation within the objective
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=fixed_params.get('random_state', 42))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        try:
            model = xgb.XGBClassifier(**params)
            model.fit(X_tr, y_tr, verbose=False)
            preds_proba = model.predict_proba(X_val)[:, 1]

            # --- Evaluate based on F1 score at a fixed threshold (e.g., 0.5) ---
            # This provides a consistent score for Optuna to optimize.
            # The final model will still undergo PTT later.
            preds_binary = (preds_proba >= 0.5).astype(int)
            f1 = f1_score(y_val, preds_binary, zero_division=0)
            cv_scores.append(f1)

        except Exception as e:
            print(f"Error during Optuna CV fold {fold} for trial {trial.number}: {e}")
            # Penalize failed trials by returning a very low score
            return 0.0 # Or handle as appropriate, e.g., np.nan and let Optuna handle

    # Return the average F1 score across folds
    average_f1 = np.mean(cv_scores) if cv_scores else 0.0
    return average_f1


# ==============================================================================
# --- Main Script Logic ---
# ==============================================================================

overall_start_time = time.time()

# --- 1. Load Data ---
print("--- Data Loading ---")
print(f"Loading data from: {CSV_FILE_PATH} (expecting SOL data without 'symbol' column)")
try:
    df_data = pd.read_csv(CSV_FILE_PATH, parse_dates=['timestamp'])
    df_data = df_data.sort_values(by='timestamp', ascending=True).reset_index(drop=True)
    print(f"Loaded {len(df_data)} rows for {SYMBOL_NAME}.")
except FileNotFoundError: print(f"Error: {CSV_FILE_PATH} not found."); exit()
except KeyError as e:
    if 'timestamp' in str(e): print(f"Error: 'timestamp' column not found in {CSV_FILE_PATH}. Please ensure it exists."); exit()
    else: print(f"Error loading data: {e}"); exit()
except Exception as e: print(f"Error loading data: {e}"); exit()


# --- Check if data is sufficient ---
if len(df_data) < TRAIN_WINDOW_MINUTES + STEP_MINUTES:
    print(f"Error: Insufficient data for {SYMBOL_NAME} ({len(df_data)} rows) for the initial training window ({TRAIN_WINDOW_MINUTES}) + step ({STEP_MINUTES}). Exiting.")
    exit()

# --- 2. Feature Engineering ---
print(f"\n--- Feature Engineering for {SYMBOL_NAME} ---")
start_fe = time.time()
df_data = calculate_features_min(df_data)
if df_data.empty: print(f"Error: Feature calculation resulted in empty DataFrame for {SYMBOL_NAME}. Exiting."); exit()
print(f"Feature engineering for {SYMBOL_NAME} complete. Took {time.time() - start_fe:.2f} seconds. Columns: {df_data.shape[1]}")

# --- 3. Define Target Variable ---
print("\n--- Target Definition ---")
print(f"Defining target as {PREDICTION_WINDOW_MINUTES}m future return >= {TARGET_THRESHOLD_PCT}%...")
target_col = f'target_return_{PREDICTION_WINDOW_MINUTES}m'
df_data[target_col] = df_data['close'].shift(-PREDICTION_WINDOW_MINUTES).sub(df_data['close']).div(df_data['close'].replace(0, np.nan)).mul(100)

# --- 4. Prepare Data for Modeling ---
print("\n--- Data Preparation ---")
base_cols_ohlcv = ['open', 'high', 'low', 'close', 'volumefrom', 'volumeto']
cols_to_keep_final = ['timestamp', target_col]
potential_feature_cols = [col for col in df_data.columns if col not in cols_to_keep_final and col not in base_cols_ohlcv]
numeric_feature_cols = df_data[potential_feature_cols].select_dtypes(include=np.number).columns.tolist()
final_feature_cols = numeric_feature_cols
cols_to_select = final_feature_cols + [col for col in cols_to_keep_final if col in df_data.columns]
df_model_ready = df_data[cols_to_select].copy()

# --- NaN / Inf Handling ---
print("Applying NaN/Inf Handling...")
initial_cols = len(final_feature_cols)
nan_threshold = 0.3
nan_percentages = df_model_ready[final_feature_cols].isna().mean()
cols_to_drop = nan_percentages[nan_percentages > nan_threshold].index.tolist()
if cols_to_drop:
    print(f"Dropping {len(cols_to_drop)} columns with >{nan_threshold*100:.0f}% NaN values: {cols_to_drop}")
    df_model_ready = df_model_ready.drop(columns=cols_to_drop)
    final_feature_cols = [col for col in final_feature_cols if col not in cols_to_drop]
initial_rows = len(df_model_ready)
df_model_ready = df_model_ready.dropna(subset=final_feature_cols + [target_col])
final_rows = len(df_model_ready)
print(f"NaN Handling: Dropped {len(cols_to_drop)} columns. Dropped {initial_rows - final_rows} rows with remaining NaNs.")
if not final_feature_cols: print(f"Error: No numeric feature columns remaining after NaN handling for {SYMBOL_NAME}. Exiting."); exit()
numeric_cols_final_check = df_model_ready[final_feature_cols].select_dtypes(include=np.number).columns.tolist()
inf_mask = np.isinf(df_model_ready[numeric_cols_final_check]); inf_count = inf_mask.sum().sum()
if inf_count > 0:
    print(f"Replacing {inf_count} infinites with NaN in features...")
    df_model_ready.replace([np.inf, -np.inf], np.nan, inplace=True)
    rows_b4 = len(df_model_ready)
    df_model_ready = df_model_ready.dropna(subset=final_feature_cols)
    print(f"Dropped {rows_b4 - len(df_model_ready)} more rows after Inf handling.")
if df_model_ready.empty: print(f"Error: DataFrame empty after NaN/Inf handling for {SYMBOL_NAME}. Exiting."); exit()

# --- Final X, y, timestamps ---
X = df_model_ready[final_feature_cols]
y_binary = (df_model_ready[target_col] >= TARGET_THRESHOLD_PCT).astype(int)
timestamps = df_model_ready['timestamp']
print(f"\nFinal feature matrix shape for {SYMBOL_NAME}: {X.shape}")
print(f"Target vector shape for {SYMBOL_NAME}: {y_binary.shape}")
print(f"Using {len(final_feature_cols)} features.")

# --- 5. SLIDING Window Backtesting (with Optuna) ---
print(f"\n--- Starting SLIDING Window Backtest for {SYMBOL_NAME} with Optuna ({N_OPTUNA_TRIALS} trials/step) ---")
if len(X) < TRAIN_WINDOW_MINUTES + STEP_MINUTES:
    print(f"Error: Not enough data for {SYMBOL_NAME} ({len(X)}) after pre-processing for train window ({TRAIN_WINDOW_MINUTES}) + step ({STEP_MINUTES}). Exiting.")
    exit()

all_predictions_proba = []; all_actual = []; backtest_timestamps = []
all_best_params = [] # Store best params found by Optuna for each step
num_steps = 0
start_index_loop = TRAIN_WINDOW_MINUTES
end_index_loop = len(X) - TEST_WINDOW_MINUTES + 1

print(f"Train Window: {TRAIN_WINDOW_MINUTES} mins, Step: {STEP_MINUTES} mins, Test Window: {TEST_WINDOW_MINUTES} mins")
loop_start_time = time.time()

for i in range(start_index_loop, end_index_loop, STEP_MINUTES):
    step_start_time = time.time()
    train_idx_start = i - TRAIN_WINDOW_MINUTES
    train_idx_end = i
    test_idx_start = i
    test_idx_end = min(i + TEST_WINDOW_MINUTES, len(X))

    if test_idx_start >= test_idx_end: print(f"Stopping loop: Test window invalid."); break

    X_train_roll = X.iloc[train_idx_start : train_idx_end]
    y_train_roll = y_binary.iloc[train_idx_start : train_idx_end]
    X_test_roll = X.iloc[test_idx_start : test_idx_end]
    y_test_roll_actual_series = y_binary.iloc[test_idx_start : test_idx_end]
    step_timestamps = timestamps.iloc[test_idx_start : test_idx_end]

    if y_test_roll_actual_series.empty: print(f"Warning: Skipping step {i} for {SYMBOL_NAME}. Empty test actuals."); continue
    current_timestamp = step_timestamps.iloc[0]
    if X_train_roll.empty or len(np.unique(y_train_roll)) < 2:
        print(f"Warning: Skipping step {i} for {SYMBOL_NAME}. Invalid training data."); continue

    print(f"\n--- {SYMBOL_NAME} - Step {num_steps + 1} (Predicting window starting {current_timestamp}) ---")
    print(f"  Training indices: [{train_idx_start}:{train_idx_end-1}]; Testing indices: [{test_idx_start}:{test_idx_end-1}]")

    # --- Hyperparameter Tuning with Optuna ---
    print(f"  Running Optuna ({N_OPTUNA_TRIALS} trials, {N_CV_FOLDS}-fold CV, maximizing F1@0.5)...")
    optuna_start_time = time.time()
    try:
        # Create study object. Maximize F1 score. Add pruner.
        study = optuna.create_study(
            direction='maximize',
            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5) # Prune after 5 trials
        )

        # Define the objective function with necessary arguments for this step
        obj_func = lambda trial: objective(trial, X_train_roll, y_train_roll, XGB_FIXED_PARAMS, N_CV_FOLDS)

        # Run optimization
        study.optimize(obj_func, n_trials=N_OPTUNA_TRIALS, timeout=None) # Set timeout (seconds) if needed

        best_params_step = study.best_params
        best_score_step = study.best_value # Best average F1@0.5 found in CV
        print(f"  Optuna finished in {time.time() - optuna_start_time:.2f}s.")
        print(f"  Best Params (suggested): {best_params_step}")
        print(f"  Best CV F1 Score (at 0.5 thresh): {best_score_step:.4f}")
        all_best_params.append({'step': num_steps + 1, 'params': best_params_step, 'cv_f1': best_score_step})

        # --- Fit final model for the step using best params found by Optuna ---
        final_model_params = {**XGB_FIXED_PARAMS, **best_params_step} # Combine fixed and best suggested
        model_roll = xgb.XGBClassifier(**final_model_params)
        model_roll.fit(X_train_roll, y_train_roll, verbose=False)

        # --- Predict probabilities for the test window ---
        prob_roll_window = model_roll.predict_proba(X_test_roll)[:, 1]

        # --- Store results ---
        all_predictions_proba.extend(prob_roll_window)
        all_actual.extend(y_test_roll_actual_series.tolist())
        backtest_timestamps.extend(step_timestamps.tolist())
        num_steps += 1

    except optuna.exceptions.TrialPruned as e:
         print(f"!! Optuna trial pruned at step {i}: {e}")
         # Decide how to handle: continue to next step or stop? Let's continue.
         continue
    except Exception as e_step:
        print(f"!! Error during Optuna study or final fit at step {i} for {SYMBOL_NAME}: {e_step}")
        traceback.print_exc() # Print stack trace for debugging
        continue # Skip to next step

    step_end_time = time.time()
    print(f"  Step {num_steps} finished in {step_end_time - step_start_time:.2f}s total.")


loop_end_time = time.time()
print(f"\nBacktesting loop for {SYMBOL_NAME} finished. Completed {num_steps} steps (each predicting up to {TEST_WINDOW_MINUTES} points) in {(loop_end_time - loop_start_time)/60:.2f} minutes.")


# --- 6. Evaluate Backtesting Results with PTT (Unchanged) ---
if num_steps > 0 and len(all_predictions_proba) == len(all_actual):
    print(f"\n--- Evaluating Results for {SYMBOL_NAME} with Probability Threshold Tuning ---")
    print(f"Threshold search range: {THRESHOLD_SEARCH_RANGE}")
    best_threshold = 0.5; best_f1_thresh = -1.0
    results_per_threshold = {}
    probabilities_np = np.array(all_predictions_proba)
    actual_np = np.array(all_actual)

    for t in THRESHOLD_SEARCH_RANGE:
        predictions_thresh = (probabilities_np >= t).astype(int)
        if np.sum(actual_np) == 0 and np.sum(predictions_thresh) == 0: acc_t, pre_t, rec_t, f1_t = 1.0, 1.0, 1.0, 1.0
        elif np.sum(actual_np) > 0 and np.sum(predictions_thresh) == 0: acc_t = accuracy_score(actual_np, predictions_thresh); pre_t, rec_t, f1_t = 0.0, 0.0, 0.0
        elif np.sum(actual_np) == 0 and np.sum(predictions_thresh) > 0: acc_t = accuracy_score(actual_np, predictions_thresh); pre_t, rec_t, f1_t = 0.0, 0.0, 0.0
        else:
             acc_t = accuracy_score(actual_np, predictions_thresh)
             pre_t = precision_score(actual_np, predictions_thresh, zero_division=0)
             rec_t = recall_score(actual_np, predictions_thresh, zero_division=0)
             f1_t = f1_score(actual_np, predictions_thresh, zero_division=0)
        results_per_threshold[round(t, 2)] = {'f1': f1_t, 'acc': acc_t, 'pre': pre_t, 'rec': rec_t}
        if f1_t >= best_f1_thresh:
             if f1_t > best_f1_thresh or abs(t - 0.5) < abs(best_threshold - 0.5):
                  best_f1_thresh = f1_t; best_threshold = t

    print(f"\nBest Threshold for {SYMBOL_NAME} found: {best_threshold:.2f} (Yielding F1 Score: {best_f1_thresh:.4f})")
    final_predictions_optimized = (probabilities_np >= best_threshold).astype(int)
    final_accuracy = accuracy_score(actual_np, final_predictions_optimized)
    final_precision = precision_score(actual_np, final_predictions_optimized, zero_division=0)
    final_recall = recall_score(actual_np, final_predictions_optimized, zero_division=0)
    final_f1 = f1_score(actual_np, final_predictions_optimized, zero_division=0)

    print(f"\n--- Final Performance Metrics for {SYMBOL_NAME} (Optimized Threshold) ---")
    print(f"Target: {PREDICTION_WINDOW_MINUTES}m return >= {TARGET_THRESHOLD_PCT}%")
    print(f"Windowing: Train={TRAIN_WINDOW_MINUTES}m, Step={STEP_MINUTES}m, Test Window={TEST_WINDOW_MINUTES}m")
    print(f"Hyperparameter Tuning: Optuna ({N_OPTUNA_TRIALS} trials/step)") # Added Optuna info
    print(f"Total Individual Predictions Evaluated: {len(actual_np)}")
    print(f"Positive Target Occurrence: {actual_np.mean()*100:.2f}%")
    print(f"Overall Accuracy:  {final_accuracy:.4f}")
    print(f"Overall Precision: {final_precision:.4f}")
    print(f"Overall Recall:    {final_recall:.4f}")
    print(f"Overall F1 Score:  {final_f1:.4f}")
    if 0.5 in results_per_threshold:
        res_def = results_per_threshold[0.5]
        print(f"(Compare: Default 0.5 Thresh -> F1:{res_def['f1']:.4f}, Acc:{res_def['acc']:.4f}, Pre:{res_def['pre']:.4f}, Rec:{res_def['rec']:.4f})")

    results_summary = { # Store results (unchanged)
        'symbol': SYMBOL_NAME,'probabilities': probabilities_np,'actuals': actual_np,'timestamps': backtest_timestamps,
        'best_threshold': best_threshold,'metrics_optimized': {'acc': final_accuracy, 'pre': final_precision, 'rec': final_recall, 'f1': final_f1},
        'metrics_default_0.5': results_per_threshold.get(0.5, {}),'best_params_per_step': all_best_params,'results_per_threshold': results_per_threshold
    }

    # --- 7. Plot Cumulative Accuracy (Unchanged) ---
    print(f"\nPlotting cumulative accuracy for {SYMBOL_NAME} (optimized threshold)...")
    if len(backtest_timestamps) != len(actual_np):
         print(f"Warning: Timestamp length ({len(backtest_timestamps)}) mismatch with prediction length ({len(actual_np)}). Skipping plot.")
    else:
        try:
            cumulative_accuracy_list_optimized = (np.cumsum(final_predictions_optimized == actual_np) / np.arange(1, len(actual_np) + 1))
            plt.figure(figsize=(14, 7))
            plt.plot(backtest_timestamps, cumulative_accuracy_list_optimized, marker='.', linestyle='-', markersize=1, alpha=0.7, label=f'Cumulative Accuracy ({SYMBOL_NAME})')
            rolling_window_plot_size = max(TEST_WINDOW_MINUTES * 5, 60 * 12)
            if len(actual_np) > rolling_window_plot_size:
                 results_df = pd.DataFrame({'correct': (final_predictions_optimized == actual_np).astype(int)}, index=pd.to_datetime(backtest_timestamps))
                 try: rolling_acc = results_df['correct'].rolling(window=rolling_window_plot_size, min_periods=rolling_window_plot_size//2).mean()
                 except Exception as e_roll: print(f"Could not calculate rolling accuracy: {e_roll}"); rolling_acc = None
                 if rolling_acc is not None: plt.plot(rolling_acc.index, rolling_acc, linestyle='--', color='red', label=f'Rolling Acc ({rolling_window_plot_size} min window)')
            plt.title(f'{SYMBOL_NAME} Backtest (Optuna Tune, Train:{TRAIN_WINDOW_MINUTES}m, Step:{STEP_MINUTES}m) - Best Thresh: {best_threshold:.2f}') # Updated title
            plt.xlabel('Timestamp'); plt.ylabel('Accuracy')
            min_y_plot = max(0.0, np.min(cumulative_accuracy_list_optimized) - 0.05 if len(cumulative_accuracy_list_optimized)>0 else 0.4)
            max_y_plot = min(1.0, np.max(cumulative_accuracy_list_optimized) + 0.05 if len(cumulative_accuracy_list_optimized)>0 else 0.8)
            if max_y_plot - min_y_plot < 0.1: mid_point=(max_y_plot+min_y_plot)/2; min_y_plot=max(0.0, mid_point-0.05); max_y_plot=min(1.0, mid_point+0.05)
            plt.ylim(min_y_plot, max_y_plot)
            plt.grid(True, linestyle='--', alpha=0.6); plt.legend(); plt.xticks(rotation=30, ha='right'); plt.tight_layout()
            plot_filename = f"backtest_accuracy_{SYMBOL_NAME}_12h_target_optuna.png" # Updated filename
            plt.savefig(plot_filename); print(f"Saved accuracy plot to {plot_filename}"); plt.close()
        except Exception as e_plot: print(f"Error plotting for {SYMBOL_NAME}: {e_plot}")
else:
    print(f"No predictions were made/stored for {SYMBOL_NAME}, cannot evaluate or plot.")

# --- End of Script ---
print(f"\n{'='*30} Overall Script Finished for {SYMBOL_NAME} {'='*30}")
overall_end_time = time.time()
print(f"Total execution time: {(overall_end_time - overall_start_time)/60:.2f} minutes.")

--- Data Loading ---
Loading data from: SOL_minagg.csv (expecting SOL data without 'symbol' column)
Loaded 22582 rows for SOL.

--- Feature Engineering for SOL ---
Feature engineering for SOL complete. Took 0.02 seconds. Columns: 48

--- Target Definition ---
Defining target as 720m future return >= 2.5%...

--- Data Preparation ---
Applying NaN/Inf Handling...
Dropping 1 columns with >30% NaN values: ['lag_10080m_price_return']
NaN Handling: Dropped 1 columns. Dropped 5041 rows with remaining NaNs.

Final feature matrix shape for SOL: (17541, 40)
Target vector shape for SOL: (17541,)
Using 40 features.

--- Starting SLIDING Window Backtest for SOL with Optuna (50 trials/step) ---
Train Window: 1440 mins, Step: 60 mins, Test Window: 288 mins

--- SOL - Step 1 (Predicting window starting 2025-04-07 03:01:00) ---
  Training indices: [120:1559]; Testing indices: [1560:1847]
  Running Optuna (50 trials, 3-fold CV, maximizing F1@0.5)...
  Optuna finished in 21.19s.
  Best Params (suggested)

[W 2025-04-18 14:18:19,776] Trial 37 failed with parameters: {'learning_rate': 0.08106390436713795, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.8631254012133971, 'colsample_bytree': 0.7624090848033295, 'scale_pos_weight': 4} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\mason\AppData\Roaming\Python\Python312\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\mason\AppData\Local\Temp\ipykernel_29944\2283494743.py", line 337, in <lambda>
    obj_func = lambda trial: objective(trial, X_train_roll, y_train_roll, XGB_FIXED_PARAMS, N_CV_FOLDS)
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mason\AppData\Local\Temp\ipykernel_29944\2283494743.py", line 187, in objective
    model.fit(X_tr, y_tr, verbose=False)
  File "C:\Users\mason\AppData\Roaming\Python

KeyboardInterrupt: 

In [4]:
X.columns

Index(['open', 'high', 'low', 'close', 'Volume BTC', 'Volume USD',
       'price_range_pct', 'oc_change_pct', 'garman_klass_12h', 'parkinson_3h',
       'ma_3h', 'rolling_std_3h', 'lag_3h_price_return', 'lag_6h_price_return',
       'lag_12h_price_return', 'lag_24h_price_return', 'lag_48h_price_return',
       'lag_72h_price_return', 'lag_168h_price_return', 'volume_return_1h',
       'lag_3h_volume_return', 'lag_6h_volume_return', 'lag_12h_volume_return',
       'lag_24h_volume_return', 'ma_6h', 'ma_12h', 'ma_24h', 'ma_48h',
       'ma_72h', 'ma_168h', 'rolling_std_6h', 'rolling_std_12h',
       'rolling_std_24h', 'rolling_std_48h', 'rolling_std_72h',
       'rolling_std_168h', 'atr_14h', 'atr_24h', 'atr_48h', 'close_div_ma_24h',
       'close_div_ma_48h', 'close_div_ma_168h', 'ma12_div_ma48',
       'ma24_div_ma168', 'std12_div_std72', 'volume_btc_x_range',
       'rolling_std_3h_sq', 'price_return_1h_sq', 'rolling_std_12h_sqrt'],
      dtype='object')